# Extrair farmácias

In [2]:
import requests
from bs4 import BeautifulSoup
import random

# Lista de headers para simular diferentes navegadores
headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-PT,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    }
]

# URL da página
url = "https://www.farmaciasdeservico.net/localidade/faro/faro"

# Escolher um header aleatoriamente
headers = random.choice(headers_list)

# Fazer a requisição HTTP
try:
    response = requests.get(url, headers=headers, timeout=10)

    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Criar o objeto BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Encontrar o elemento com id="conteudoFarmacias"
        conteudo_farmacias = soup.find(id='conteudoFarmacias')
        
        if conteudo_farmacias:
            # Extrair todo o conteúdo dentro do elemento
            with open('farmacias_faro.html', 'w', encoding='utf-8') as file:
                file.write(conteudo_farmacias.prettify())
            print("Conteúdo extraído e salvo em 'farmacias_faro.html'.")
            
            # Opcional: Imprimir o conteúdo no console
            print(conteudo_farmacias.prettify())
        else:
            print("Elemento com id='conteudoFarmacias' não encontrado.")
    else:
        print(f"Erro na requisição: Status code {response.status_code}")
except requests.RequestException as e:
    print(f"Erro ao fazer a requisição: {e}")

Conteúdo extraído e salvo em 'farmacias_faro.html'.
<div id="conteudoFarmacias">
 <section class="blocoFarm cP">
  <h2 class="separadorTipo cP sticky">
   <strong>
    Permanente
   </strong>
   <em>
    Aberta 24h
   </em>
  </h2>
  <div class="farmacia destaque" data-horario="1748559600000-1748646000000">
   <a class="mapa" data-modal="iframe" href="mapa/9814" title="Localização">
    
   </a>
   <a class="logoFarmacia" href="f/9814/Farmacia_Almeida/">
    <img alt="Logótipo da Farmácia Almeida" decoding="async" loading="lazy" src="logos/9814.png"/>
   </a>
   <ul class="botoes">
    <li class="telefone">
     <a class="botao icone telefone" href="tel:289822437" onclick="stat(9814, 'telefone')">
      <span>
       289 822 437
      </span>
     </a>
    </li>
    <li>
     <a alt="http://www.farmacia-almeida.com" class="botao icone website" href="http://www.farmacia-almeida.com" onclick="stat(9814, 'website')" rel="nofollow noopener noreferrer" target="_blank">
      Website
     </

In [9]:
from bs4 import BeautifulSoup
import json

# Ler o arquivo HTML salvo
with open('farmacias_faro.html', 'r', encoding='utf-8') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Localizar o elemento com id="conteudoFarmacias"
conteudo_farmacias = soup.find(id='conteudoFarmacias')

# Lista para armazenar os dados extraídos
farmacias = []

# Função para extrair horários
def extrair_horario(horario_div):
    horarios = []
    for linha in horario_div.find_all('div', class_='linha'):
        inicio = linha.find_all('strong')[0].text.strip()
        fim = linha.find_all('strong')[1].text.strip()
        horarios.append(f"{inicio} às {fim}")
    return horarios

# Função para extrair serviços
def extrair_servicos(servicos_div):
    if servicos_div:
        return [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']
    return []

# Iterar sobre as seções (Permanente, Horário alargado, Disponibilidade)
for secao in conteudo_farmacias.find_all('section', class_='blocoFarm'):
    tipo = secao.find('h2', class_='separadorTipo').find('strong').text.strip()
    
    # Iterar sobre as farmácias na seção
    for farmacia in secao.find_all('div', class_=['farmacia destaque', 'farmacia normal']):
        dados = farmacia.find('div', class_='dados')
        nome_morada = dados.find('div', class_='nomeMorada')
        
        # Extrair informações
        nome = nome_morada.find('h3').find('a').text.strip()
        link_farmacia = nome_morada.find('h3').find('a')['href']
        morada = ' '.join(nome_morada.find('p', class_='morada').text.split())
        
        # Extrair telefone
        telefone = dados.find('a', class_='botao icone telefone')
        telefone = telefone.find('span').text.strip() if telefone else 'N/A'
        
        # Extrair horário
        horario_div = dados.find('div', class_='horario')
        horarios = extrair_horario(horario_div) if horario_div else []
        
        # Extrair serviços (se disponíveis)
        servicos_div = dados.find('div', class_='servicos')
        servicos = extrair_servicos(servicos_div)
        
        # Extrair website (se disponível)
        website = farmacia.find('a', class_='botao icone website')
        website = website['href'] if website else 'N/A'
        
        # Extrair link do mapa
        mapa = farmacia.find('a', class_='mapa')
        link_mapa = mapa['href'] if mapa else 'N/A'
        
        # Armazenar os dados
        farmacias.append({
            'tipo': tipo,
            'nome': nome,
            'morada': morada,
            'telefone': telefone,
            'horarios': horarios,
            'servicos': servicos,
            'website': website,
            'link_mapa': link_mapa,
            'link_farmacia': link_farmacia
        })

# Salvar os dados em JSON
with open('farmacias_faro.json', 'w', encoding='utf-8') as file:
    json.dump(farmacias, file, ensure_ascii=False, indent=4)

print("Dados extraídos e salvos em 'farmacias_faro.json'.")

Dados extraídos e salvos em 'farmacias_faro.json'.


In [14]:
import json
from datetime import datetime, time
from tabulate import tabulate

# Função para verificar se a farmácia está aberta às 12:49
def esta_aberta(horarios, hora_atual=time(12, 49)):
    for horario in horarios:
        inicio, fim = horario.split(' às ')
        inicio_h, inicio_m = map(int, inicio.split(':'))
        
        # Tratar o caso especial de "24:00"
        if fim == "24:00":
            fim_h, fim_m = 23, 59
        else:
            fim_h, fim_m = map(int, fim.split(':'))
        
        inicio_t = time(inicio_h, inicio_m)
        fim_t = time(fim_h, fim_m)
        
        if inicio_t <= hora_atual <= fim_t:
            return True
    return False

# Ler o arquivo JSON
try:
    with open('farmacias_faro.json', 'r', encoding='utf-8') as file:
        farmacias = json.load(file)
except FileNotFoundError:
    print("Arquivo 'farmacias_faro.json' não encontrado. Execute o script anterior para gerá-lo.")
    exit()

# Filtrar farmácias abertas às 12:49 PM
farmacias_abertas = [f for f in farmacias if esta_aberta(f['horarios'])]

# Preparar dados para a tabela
tabela = []
for f in farmacias_abertas:
    tabela.append([
        f['nome'],
        f['tipo'],
        f['morada'],
        f['telefone'],
        ', '.join(f['horarios']),
        ', '.join(f['servicos']) if f['servicos'] else 'N/A',
        f['website'],
        f['link_mapa']
    ])

# Definir cabeçalhos da tabela
headers = ['Nome', 'Tipo', 'Morada', 'Telefone', 'Horários', 'Serviços', 'Website', 'Link Mapa']

# Exibir a tabela com tabulate
print("\nFarmácias abertas às 12:49 PM WEST (30/05/2025):")
print(tabulate(tabela, headers=headers, tablefmt='grid'))

# Informar o número de farmácias encontradas
print(f"\nTotal de farmácias abertas: {len(farmacias_abertas)}")


Farmácias abertas às 12:49 PM WEST (30/05/2025):
+------------------------+------------------+---------------------------------------------------------------------------------+-------------+--------------------------------+------------------------------------------------------------------------------+-------------------------------------------+-------------+
| Nome                   | Tipo             | Morada                                                                          | Telefone    | Horários                       | Serviços                                                                     | Website                                   | Link Mapa   |
+========================+==================+=================================================================================+=============+================================+==============================================================================+===========================================+=============+
| Farmácia Alm

In [16]:
from bs4 import BeautifulSoup
from datetime import datetime, time
from tabulate import tabulate

# HTML fornecido
html = '''
<div class="farmacia destaque" data-horario="1748559600000-1748646000000"><a href="mapa/10693" data-modal="iframe" title="Localização" class="mapa"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mNkYAAAAAYAAjCB0C8AAAAASUVORK5CYII=" alt="Mapa com a localização da Farmácia da Penha" style="background-image: url('imgs/mapa/37.023685/-7.926673/838/118/2.png')"></a><a href="f/10693/Farmacia_da_Penha/" class="logoFarmacia"><img alt="Logótipo da Farmácia da Penha" src="logos/10693.png" loading="lazy" decoding="async"></a><ul class="botoes"><li class="telefone"><a href="tel:289822342" class="botao icone telefone" onclick="stat(10693, 'telefone')"><span>289 822 342</span></a></li><li><a href="http://www.farmaciadapenha.pt" alt="http://www.farmaciadapenha.pt" rel="nofollow noopener noreferrer" class="botao icone website" target="_blank" onclick="stat(10693, 'website')">Website</a></li><li><a href="email/10693" class="botao icone email" data-modal="ajax">Email</a></li><li><a href="mapa/10693" data-modal="iframe" title="Localização" class="botao icone bmapa">Mapa</a></li></ul><div class="dados"><div class="nomeMorada"><h3><a href="f/10693/Farmacia_da_Penha/">Farmácia da Penha</a></h3><p class="morada">Estrada da Penha, 52<br>Sé</p></div><div class="horario">
                        <h4 title="Horário">Horário</h4>
                        <div class="linha"><strong>00:00</strong> às <strong>24:00</strong></div>
                    </div><div class="servicos">
								<h4>Serviços</h4>
								<ul><li alt="Entregas ao Domicílio" title="Entregas ao Domicílio" class="s11">Entregas ao Domicílio</li><li alt="Administração de Vacinas" title="Administração de Vacinas" class="s34">Administração de Vacinas</li><li alt="Medição de Pressão Arterial" title="Medição de Pressão Arterial" class="s8">Medição de Pressão Arterial</li><li>…</li></ul><a href="f/10693/Farmacia_da_Penha/" alt="Ver todos"><span></span></a></div></div><a href="f/10693/Farmacia_da_Penha/" class="mobile"><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/wIAAgMBAp9lfnQAAAAASUVORK5CYII=" alt="Farmácia da Penha"></a>
				</div>
'''

# Criar o objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Função para verificar se a farmácia está aberta às 12:53
def esta_aberta(horarios, hora_atual=time(12, 53)):
    for horario in horarios:
        inicio, fim = horario.split(' às ')
        inicio_h, inicio_m = map(int, inicio.split(':'))
        # Tratar o caso especial de "24:00"
        if fim == "24:00":
            fim_h, fim_m = 23, 59
        else:
            fim_h, fim_m = map(int, fim.split(':'))
        inicio_t = time(inicio_h, inicio_m)
        fim_t = time(fim_h, fim_m)
        if inicio_t <= hora_atual <= fim_t:
            return True
    return False

# Extrair informações da farmácia
farmacia = soup.find('div', class_='farmacia destaque')

# Nome
nome = farmacia.find('h3').find('a').text.strip()

# Tipo (derivado do contexto, não presente diretamente no trecho, assumido como "Permanente" pelo horário 24h)
tipo = "Permanente"

# Morada
morada = ' '.join(farmacia.find('p', class_='morada').text.split())

# Telefone
telefone = farmacia.find('a', class_='botao icone telefone').find('span').text.strip()

# Horários
horario_div = farmacia.find('div', class_='horario')
horarios = []
for linha in horario_div.find_all('div', class_='linha'):
    inicio = linha.find_all('strong')[0].text.strip()
    fim = linha.find_all('strong')[1].text.strip()
    horarios.append(f"{inicio} às {fim}")

# Serviços
servicos_div = farmacia.find('div', class_='servicos')
servicos = [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']

# Website
website = farmacia.find('a', class_='botao icone website')['href']

# Link do mapa
link_mapa = farmacia.find('a', class_='mapa')['href']

# Link da farmácia
link_farmacia = farmacia.find('h3').find('a')['href']

# Coordenadas do mapa (extraídas do estilo da imagem)
mapa_img = farmacia.find('a', class_='mapa').find('img')['style']
coordenadas = mapa_img.split('url(\'imgs/mapa/')[1].split('/')[0]

# Timestamps (data-horario)
data_horario = farmacia['data-horario']
inicio_ts, fim_ts = map(int, data_horario.split('-'))
inicio_dt = datetime.fromtimestamp(inicio_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
fim_dt = datetime.fromtimestamp(fim_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')

# Verificar se está aberta às 12:53
aberta = esta_aberta(horarios)

# Montar dicionário com os detalhes
farmacia_detalhes = {
    'Nome': nome,
    'Tipo': tipo,
    'Morada': morada,
    'Telefone': telefone,
    'Horários': ', '.join(horarios),
    'Serviços': ', '.join(servicos) if servicos else 'N/A',
    'Website': website,
    'Link Mapa': link_mapa,
    'Link Farmácia': link_farmacia,
    'Coordenadas': coordenadas,
    'Timestamp Início': inicio_dt,
    'Timestamp Fim': fim_dt,
    'Aberta às 12:53': 'Sim' if aberta else 'Não'
}

# Preparar dados para a tabela
tabela = [[k, v] for k, v in farmacia_detalhes.items()]

# Exibir tabela com tabulate
print("\nDetalhes da Farmácia da Penha:")
print(tabulate(tabela, headers=['Campo', 'Valor'], tablefmt='grid'))


Detalhes da Farmácia da Penha:
+------------------+------------------------------------------------------------------------------+
| Campo            | Valor                                                                        |
+==================+==============================================================================+
| Nome             | Farmácia da Penha                                                            |
+------------------+------------------------------------------------------------------------------+
| Tipo             | Permanente                                                                   |
+------------------+------------------------------------------------------------------------------+
| Morada           | Estrada da Penha, 52Sé                                                       |
+------------------+------------------------------------------------------------------------------+
| Telefone         | 289 822 342                                    

In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time
from tabulate import tabulate
import random

# Lista de headers para simular diferentes navegadores
headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-PT,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    }
]

# URL do site
url = "https://www.farmaciasdeservico.net/localidade/faro/faro"

# Função para verificar se a farmácia está aberta às 12:54
def esta_aberta(horarios, hora_atual=time(12, 54)):
    for horario in horarios:
        inicio, fim = horario.split(' às ')
        inicio_h, inicio_m = map(int, inicio.split(':'))
        # Tratar o caso especial de "24:00"
        if fim == "24:00":
            fim_h, fim_m = 23, 59
        else:
            fim_h, fim_m = map(int, fim.split(':'))
        inicio_t = time(inicio_h, inicio_m)
        fim_t = time(fim_h, fim_m)
        if inicio_t <= hora_atual <= fim_t:
            return True
    return False

# Função para extrair horários
def extrair_horario(horario_div):
    horarios = []
    for linha in horario_div.find_all('div', class_='linha'):
        inicio = linha.find_all('strong')[0].text.strip()
        fim = linha.find_all('strong')[1].text.strip()
        horarios.append(f"{inicio} às {fim}")
    return horarios

# Função para extrair serviços
def extrair_servicos(servicos_div):
    if servicos_div:
        return [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']
    return []

# Fazer a requisição HTTP
try:
    headers = random.choice(headers_list)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Erro ao acessar o site: {e}")
    exit()

# Criar o objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Localizar o elemento com id="conteudoFarmacias"
conteudo_farmacias = soup.find(id='conteudoFarmacias')
if not conteudo_farmacias:
    print("Elemento com id='conteudoFarmacias' não encontrado.")
    exit()

# Lista para armazenar os dados das farmácias
farmacias = []

# Iterar sobre as seções (Permanente, Horário alargado, Disponibilidade)
for secao in conteudo_farmacias.find_all('section', class_='blocoFarm'):
    tipo = secao.find('h2', class_='separadorTipo').find('strong').text.strip()
    
    # Iterar sobre as farmácias na seção
    for farmacia in secao.find_all('div', class_=['farmacia destaque', 'farmacia normal']):
        dados = farmacia.find('div', class_='dados')
        nome_morada = dados.find('div', class_='nomeMorada')
        
        # Extrair informações
        nome = nome_morada.find('h3').find('a').text.strip()
        link_farmacia = nome_morada.find('h3').find('a')['href']
        morada = ' '.join(nome_morada.find('p', class_='morada').text.split())
        
        # Extrair telefone
        telefone = dados.find('a', class_='botao icone telefone')
        telefone = telefone.find('span').text.strip() if telefone else 'N/A'
        
        # Extrair horário
        horario_div = dados.find('div', class_='horario')
        horarios = extrair_horario(horario_div) if horario_div else []
        
        # Extrair serviços
        servicos_div = dados.find('div', class_='servicos')
        servicos = extrair_servicos(servicos_div)
        
        # Extrair website
        website = farmacia.find('a', class_='botao icone website')
        website = website['href'] if website else 'N/A'
        
        # Extrair link do mapa
        mapa = farmacia.find('a', class_='mapa')
        link_mapa = mapa['href'] if mapa else 'N/A'
        
        # Extrair coordenadas (se disponíveis)
        coordenadas = 'N/A'
        if mapa and mapa.find('img') and 'style' in mapa.find('img').attrs:
            mapa_img = mapa.find('img')['style']
            if 'url(\'imgs/mapa/' in mapa_img:
                coordenadas = mapa_img.split('url(\'imgs/mapa/')[1].split('/')[0]
        
        # Extrair timestamps (data-horario)
        data_horario = farmacia.get('data-horario', 'N/A')
        inicio_dt = fim_dt = 'N/A'
        if data_horario != 'N/A':
            try:
                inicio_ts, fim_ts = map(int, data_horario.split('-'))
                inicio_dt = datetime.fromtimestamp(inicio_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
                fim_dt = datetime.fromtimestamp(fim_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                inicio_dt = fim_dt = 'N/A'
        
        # Verificar se está aberta às 12:54
        aberta = esta_aberta(horarios)
        
        # Armazenar os dados
        farmacias.append({
            'nome': nome,
            'tipo': tipo,
            'morada': morada,
            'telefone': telefone,
            'horarios': horarios,
            'servicos': servicos,
            'website': website,
            'link_mapa': link_mapa,
            'link_farmacia': link_farmacia,
            'coordenadas': coordenadas,
            'timestamp_inicio': inicio_dt,
            'timestamp_fim': fim_dt,
            'aberta_12_54': 'Sim' if aberta else 'Não'
        })

# Filtrar farmácias abertas às 12:54
farmacias_abertas = [f for f in farmacias if f['aberta_12_54'] == 'Sim']

# Preparar dados para a tabela
tabela = []
for f in farmacias_abertas:
    tabela.append([
        f['nome'],
        f['tipo'],
        f['morada'],
        f['telefone'],
        ', '.join(f['horarios']),
        ', '.join(f['servicos']) if f['servicos'] else 'N/A',
        f['website'],
        f['link_mapa'],
        f['coordenadas'],
        f['timestamp_inicio'],
        f['timestamp_fim']
    ])

# Definir cabeçalhos da tabela
headers = ['Nome', 'Tipo', 'Morada', 'Telefone', 'Horários', 'Serviços', 'Website', 'Link Mapa', 'Coordenadas', 'Início', 'Fim']

# Exibir a tabela com tabulate
print("\nFarmácias abertas às 12:54 PM WEST (30/05/2025):")
print(tabulate(tabela, headers=headers, tablefmt='grid'))

# Informar o número de farmácias encontradas
print(f"\nTotal de farmácias abertas: {len(farmacias_abertas)}")


Farmácias abertas às 12:54 PM WEST (30/05/2025):
+------------------------+------------------+--------------------------------------------------------------------------------+-------------+--------------------------------+------------------------------------------------------------------------------+-------------------------------------------+-------------+---------------+---------------------+---------------------+
| Nome                   | Tipo             | Morada                                                                         | Telefone    | Horários                       | Serviços                                                                     | Website                                   | Link Mapa   |   Coordenadas | Início              | Fim                 |
+========================+==================+================================================================================+=============+================================+===================================

In [26]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time
from tabulate import tabulate
import random

# Lista de headers para simular diferentes navegadores
headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-PT,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    }
]

# URL do site
url = "https://www.farmaciasdeservico.net/localidade/faro/faro"

# Função para verificar se a farmácia está aberta às 12:55
def esta_aberta(horarios, hora_atual=time(12, 55)):
    for horario in horarios:
        inicio, fim = horario.split(' às ')
        inicio_h, inicio_m = map(int, inicio.split(':'))
        # Tratar o caso especial de "24:00"
        if fim == "24:00":
            fim_h, fim_m = 23, 59
        else:
            fim_h, fim_m = map(int, fim.split(':'))
        inicio_t = time(inicio_h, inicio_m)
        fim_t = time(fim_h, fim_m)
        if inicio_t <= hora_atual <= fim_t:
            return True
    return False

# Função para extrair horários
def extrair_horario(horario_div):
    horarios = []
    for linha in horario_div.find_all('div', class_='linha'):
        inicio = linha.find_all('strong')[0].text.strip()
        fim = linha.find_all('strong')[1].text.strip()
        horarios.append(f"{inicio} às {fim}")
    return horarios

# Função para extrair serviços
def extrair_servicos(servicos_div):
    if servicos_div:
        return [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']
    return []

# Fazer a requisição HTTP
try:
    headers = random.choice(headers_list)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Erro ao acessar o site: {e}")
    exit()

# Criar o objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Localizar o elemento com id="conteudoFarmacias"
conteudo_farmacias = soup.find(id='conteudoFarmacias')
if not conteudo_farmacias:
    print("Elemento com id='conteudoFarmacias' não encontrado.")
    exit()

# Lista para armazenar os dados das farmácias
farmacias = []

# Iterar sobre as seções (Permanente, Horário alargado, Disponibilidade)
for secao in conteudo_farmacias.find_all('section', class_='blocoFarm'):
    tipo = secao.find('h2', class_='separadorTipo').find('strong').text.strip()
    
    # Iterar sobre as farmácias na seção
    for farmacia in secao.find_all('div', class_=['farmacia destaque', 'farmacia normal']):
        dados = farmacia.find('div', class_='dados')
        nome_morada = dados.find('div', class_='nomeMorada')
        
        # Extrair informações
        nome = nome_morada.find('h3').find('a').text.strip()
        link_farmacia = nome_morada.find('h3').find('a')['href']
        morada = ' '.join(nome_morada.find('p', class_='morada').text.split())
        
        # Extrair telefone (corrigido para farmácias "destaque" e "normal")
        telefone = 'N/A'
        telefone_elem = farmacia.find('a', class_='botao icone telefone')
        if telefone_elem and telefone_elem.find('span'):
            telefone = telefone_elem.find('span').text.strip()
        
        # Extrair horário
        horario_div = dados.find('div', class_='horario')
        horarios = extrair_horario(horario_div) if horario_div else []
        
        # Extrair serviços
        servicos_div = dados.find('div', class_='servicos')
        servicos = extrair_servicos(servicos_div)
        
        # Extrair website
        website = farmacia.find('a', class_='botao icone website')
        website = website['href'] if website else 'N/A'
        
        # Extrair link do mapa
        mapa = farmacia.find('a', class_='mapa')
        link_mapa = mapa['href'] if mapa else 'N/A'
        
        # Extrair coordenadas
        coordenadas = 'N/A'
        if mapa and mapa.find('img') and 'style' in mapa.find('img').attrs:
            mapa_img = mapa.find('img')['style']
            if 'url(\'imgs/mapa/' in mapa_img:
                coordenadas = mapa_img.split('url(\'imgs/mapa/')[1].split('/')[0]
        
        # Extrair timestamps
        data_horario = farmacia.get('data-horario', 'N/A')
        inicio_dt = fim_dt = 'N/A'
        if data_horario != 'N/A':
            try:
                inicio_ts, fim_ts = map(int, data_horario.split('-'))
                inicio_dt = datetime.fromtimestamp(inicio_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
                fim_dt = datetime.fromtimestamp(fim_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                inicio_dt = fim_dt = 'N/A'
        
        # Verificar se está aberta às 12:55
        aberta = esta_aberta(horarios)
        
        # Armazenar os dados
        farmacias.append({
            'nome': nome,
            'tipo': tipo,
            'morada': morada,
            'telefone': telefone,
            'horarios': horarios,
            'servicos': servicos,
            'website': website,
            'link_mapa': link_mapa,
            'coordenadas': coordenadas,
            'timestamp_inicio': inicio_dt,
            'timestamp_fim': fim_dt,
            'aberta_12_55': 'Sim' if aberta else 'Não'
        })

# Filtrar farmácias abertas às 12:55
farmacias_abertas = [f for f in farmacias if f['aberta_12_55'] == 'Sim']

# Preparar dados para a tabela
tabela = []
for f in farmacias_abertas:
    tabela.append([
        f['nome'],
        f['tipo'],
        f['morada'],
        f['telefone'],
        ', '.join(f['horarios']),
        ', '.join(f['servicos']) if f['servicos'] else 'N/A',
        f['website'],
        f['link_mapa'],
        f['coordenadas']
        #f['timestamp_inicio'],
        #f['timestamp_fim']
    ])

# Definir cabeçalhos da tabela
headers = ['Nome', 'Tipo', 'Morada', 'Telefone', 'Horários', 'Serviços', 'Website', 'Link Mapa', 'Coordenadas', 'Início', 'Fim']

# Exibir a tabela com tabulate
print("\nFarmácias abertas às 12:55 PM WEST (30/05/2025):")
print(tabulate(tabela, headers=headers, tablefmt='grid'))

# Informar o número de farmácias encontradas
print(f"\nTotal de farmácias abertas: {len(farmacias_abertas)}")


Farmácias abertas às 12:55 PM WEST (30/05/2025):
+------------------------+------------------+--------------------------------------------------------------------------------+-------------+--------------------------------+------------------------------------------------------------------------------+-------------------------------------------+-------------+---------------+
| Nome                   | Tipo             | Morada                                                                         | Telefone    | Horários                       | Serviços                                                                     | Website                                   | Link Mapa   |   Coordenadas |
+========================+==================+================================================================================+=============+================================+==============================================================================+===========================================+

In [28]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time
from tabulate import tabulate
import random

# Lista de headers para simular diferentes navegadores
headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-PT,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    }
]

# URL do site
url = "https://www.farmaciasdeservico.net/localidade/faro/faro"

# Função para verificar se a farmácia está aberta às 01:05 PM
def esta_aberta(horarios, hora_atual=time(13, 5)):
    for horario in horarios:
        inicio, fim = horario.split(' às ')
        inicio_h, inicio_m = map(int, inicio.split(':'))
        # Tratar o caso especial de "24:00"
        if fim == "24:00":
            fim_h, fim_m = 23, 59
        else:
            fim_h, fim_m = map(int, fim.split(':'))
        inicio_t = time(inicio_h, inicio_m)
        fim_t = time(fim_h, fim_m)
        if inicio_t <= hora_atual <= fim_t:
            return True
    return False

# Função para extrair horários
def extrair_horario(horario_div):
    horarios = []
    for linha in horario_div.find_all('div', class_='linha'):
        inicio = linha.find_all('strong')[0].text.strip()
        fim = linha.find_all('strong')[1].text.strip()
        horarios.append(f"{inicio} às {fim}")
    return horarios

# Função para extrair serviços
def extrair_servicos(servicos_div):
    if servicos_div:
        return [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']
    return []

# Fazer a requisição HTTP
try:
    headers = random.choice(headers_list)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Erro ao acessar o site: {e}")
    exit()

# Criar o objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Localizar o elemento com id="conteudoFarmacias"
conteudo_farmacias = soup.find(id='conteudoFarmacias')
if not conteudo_farmacias:
    print("Elemento com id='conteudoFarmacias' não encontrado.")
    exit()

# Lista para armazenar os dados das farmácias
farmacias = []

# Iterar sobre as seções (Permanente, Horário alargado, Disponibilidade)
for secao in conteudo_farmacias.find_all('section', class_='blocoFarm'):
    tipo = secao.find('h2', class_='separadorTipo').find('strong').text.strip()
    
    # Iterar sobre as farmácias na seção
    for farmacia in secao.find_all('div', class_=['farmacia destaque', 'farmacia normal']):
        dados = farmacia.find('div', class_='dados')
        nome_morada = dados.find('div', class_='nomeMorada')
        
        # Extrair informações
        nome = nome_morada.find('h3').find('a').text.strip()
        link_farmacia = nome_morada.find('h3').find('a')['href']
        
        # Extrair morada, tratando quebras de linha
        morada_elem = nome_morada.find('p', class_='morada')
        # Substituir <br> por espaço e normalizar espaços
        morada_parts = [part.strip() for part in morada_elem.get_text(separator=' ').split()]
        morada = ' '.join(morada_parts)
        
        # Extrair telefone
        telefone = 'N/A'
        telefone_elem = farmacia.find('a', class_='botao icone telefone')
        if telefone_elem and telefone_elem.find('span'):
            telefone = telefone_elem.find('span').text.strip()
        
        # Extrair horário
        horario_div = dados.find('div', class_='horario')
        horarios = extrair_horario(horario_div) if horario_div else []
        
        # Extrair serviços
        servicos_div = dados.find('div', class_='servicos')
        servicos = extrair_servicos(servicos_div)
        
        # Extrair website
        website = farmacia.find('a', class_='botao icone website')
        website = website['href'] if website else 'N/A'
        
        # Extrair link do mapa
        mapa = farmacia.find('a', class_='mapa')
        link_mapa = mapa['href'] if mapa else 'N/A'
        
        # Extrair coordenadas
        coordenadas = 'N/A'
        if mapa and mapa.find('img') and 'style' in mapa.find('img').attrs:
            mapa_img = mapa.find('img')['style']
            if 'url(\'imgs/mapa/' in mapa_img:
                coordenadas = mapa_img.split('url(\'imgs/mapa/')[1].split('/')[0]
        
        # Extrair timestamps
        data_horario = farmacia.get('data-horario', 'N/A')
        inicio_dt = fim_dt = 'N/A'
        if data_horario != 'N/A':
            try:
                inicio_ts, fim_ts = map(int, data_horario.split('-'))
                inicio_dt = datetime.fromtimestamp(inicio_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
                fim_dt = datetime.fromtimestamp(fim_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                inicio_dt = fim_dt = 'N/A'
        
        # Verificar se está aberta às 01:05 PM
        aberta = esta_aberta(horarios)
        
        # Armazenar os dados
        farmacias.append({
            'nome': nome,
            'tipo': tipo,
            'morada': morada,
            'telefone': telefone,
            'horarios': horarios,
            'servicos': servicos,
            'website': website,
            'link_mapa': link_mapa,
            'coordenadas': coordenadas,
            'timestamp_inicio': inicio_dt,
            'timestamp_fim': fim_dt,
            'aberta_13_05': 'Sim' if aberta else 'Não'
        })

# Filtrar farmácias abertas às 01:05 PM
farmacias_abertas = [f for f in farmacias if f['aberta_13_05'] == 'Sim']

# Preparar dados para a tabela
tabela = []
for f in farmacias_abertas:
    tabela.append([
        f['nome'],
        f['tipo'],
        f['morada'],
        f['telefone'],
        ', '.join(f['horarios']),
        ', '.join(f['servicos']) if f['servicos'] else 'N/A',
        f['website'],
        f['link_mapa'],
        f['coordenadas']
    ])

# Definir cabeçalhos da tabela
headers = ['Nome', 'Tipo', 'Morada', 'Telefone', 'Horários', 'Serviços', 'Website', 'Link Mapa', 'Coordenadas', 'Início', 'Fim']

# Exibir a tabela com tabulate
print("\nFarmácias abertas às 01:05 PM WEST (30/05/2025):")
print(tabulate(tabela, headers=headers, tablefmt='grid'))

# Informar o número de farmácias encontradas
print(f"\nTotal de farmácias abertas: {len(farmacias_abertas)}")


Farmácias abertas às 01:05 PM WEST (30/05/2025):
+------------------------+------------------+---------------------------------------------------------------------------------+-------------+----------------+------------------------------------------------------------------------------+-------------------------------------------+-------------+---------------+
| Nome                   | Tipo             | Morada                                                                          | Telefone    | Horários       | Serviços                                                                     | Website                                   | Link Mapa   |   Coordenadas |
+========================+==================+=================================================================================+=============+================+==============================================================================+===========================================+=============+===============+
| Farmácia Alm

In [38]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time
from tabulate import tabulate
import random

# Lista de headers para simular diferentes navegadores
headers_list = [
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "pt-PT,pt;q=0.9,en;q=0.8",
        "Connection": "keep-alive"
    },
    {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive"
    }
]

# URL do site
url = "https://www.farmaciasdeservico.net/localidade/faro/faro"

# Função para verificar se a farmácia está aberta às 01:15 PM
def esta_aberta(horarios, hora_atual=time(13, 15)):
    for horario in horarios:
        try:
            inicio, fim = horario.strip().split(' às ')
            inicio_h, inicio_m = map(int, inicio.split(':'))
            # Tratar o caso especial de "24:00"
            if fim == "24:00":
                fim_h, fim_m = 23, 59
            else:
                fim_h, fim_m = map(int, fim.split(':'))
            inicio_t = time(inicio_h, inicio_m)
            fim_t = time(fim_h, fim_m)
            if inicio_t <= hora_atual <= fim_t:
                return True
        except ValueError as e:
            print(f"Erro ao processar horário '{horario}': {e}")
            continue
    return False

# Função para extrair horários
def extrair_horario(horario_div):
    horarios = []
    for linha in horario_div.find_all('div', class_='linha'):
        try:
            inicio = linha.find_all('strong')[0].text.strip()
            fim = linha.find_all('strong')[1].text.strip()
            horarios.append(f"{inicio} às {fim}")
        except:
            continue
    return horarios

# Função para extrair serviços
def extrair_servicos(servicos_div):
    if servicos_div:
        return [li.text.strip() for li in servicos_div.find('ul').find_all('li') if li.text.strip() != '…']
    return []

# Fazer a requisição HTTP
try:
    headers = random.choice(headers_list)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()
except requests.RequestException as e:
    print(f"Erro ao acessar o site: {e}")
    exit()

# Criar o objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Localizar o elemento com id="conteudoFarmacias"
conteudo_farmacias = soup.find(id='conteudoFarmacias')
if not conteudo_farmacias:
    print("Elemento com id='conteudoFarmacias' não encontrado.")
    exit()

# Lista para armazenar os dados das farmácias
farmacias = []

# Iterar sobre as seções (Permanente, Horário alargado, Disponibilidade)
for secao in conteudo_farmacias.find_all('section', class_='blocoFarm'):
    tipo = secao.find('h2', class_='separadorTipo').find('strong').text.strip()
    
    # Iterar sobre as farmácias na seção
    for idx, farmacia in enumerate(secao.find_all('div', class_=['farmacia destaque', 'farmacia normal']), 1):
        dados = farmacia.find('div', class_='dados')
        nome_morada = dados.find('div', class_='nomeMorada')
        
        # Extrair informações
        nome = nome_morada.find('h3').find('a').text.strip()
        link_farmacia = nome_morada.find('h3').find('a')['href']
        
        # Extrair morada, tratando quebras de linha
        morada_elem = nome_morada.find('p', class_='morada')
        morada_parts = [part.strip() for part in morada_elem.get_text(separator=' ').split()]
        morada = ' '.join(morada_parts)
        
        # Extrair telefone
        telefone = 'N/A'
        telefone_elem = farmacia.find('a', class_='botao icone telefone')
        if telefone_elem and telefone_elem.find('span'):
            telefone = telefone_elem.find('span').text.strip()
        
        # Extrair horário
        horario_div = dados.find('div', class_='horario')
        horarios = extrair_horario(horario_div) if horario_div else []
        
        # Extrair serviços
        servicos_div = dados.find('div', class_='servicos')
        servicos = extrair_servicos(servicos_div)
        
        # Extrair website
        website = farmacia.find('a', class_='botao icone website')
        website = website['href'] if website else 'N/A'
        
        # Extrair link do mapa
        mapa = farmacia.find('a', class_='mapa')
        link_mapa = mapa['href'] if mapa else 'N/A'
        
        # Extrair coordenadas
        coordenadas = 'N/A'
        if tipo == "Permanente" and idx == 2:  # Segunda farmácia da seção Permanente (Farmácia da Penha)
            mapa_img = soup.select_one('#conteudoFarmacias > section.blocoFarm.cP > div:nth-child(2) > a.mapa > img')
            if mapa_img and 'style' in mapa_img.attrs:
                mapa_style = mapa_img['style']
                if 'url(\'imgs/mapa/' in mapa_style:
                    # Capturar até a próxima barra
                    start = mapa_style.find('url(\'imgs/mapa/') + len('url(\'imgs/mapa/')
                    end = mapa_style.find('/', start)
                    coordenadas = mapa_style[start:end]
        elif mapa and mapa.find('img') and 'style' in mapa.find('img').attrs:
            mapa_img = mapa.find('img')['style']
            if 'url(\'imgs/mapa/' in mapa_img:
                # Capturar até a próxima barra
                start = mapa_img.find('url(\'imgs/mapa/') + len('url(\'imgs/mapa/')
                end = mapa_img.find('/', start)
                coordenadas = mapa_img[start:end]
        
        # Extrair timestamps
        data_horario = farmacia.get('data-horario', 'N/A')
        inicio_dt = fim_dt = 'N/A'
        if data_horario != 'N/A':
            try:
                inicio_ts, fim_ts = map(int, data_horario.split('-'))
                inicio_dt = datetime.fromtimestamp(inicio_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
                fim_dt = datetime.fromtimestamp(fim_ts / 1000).strftime('%Y-%m-%d %H:%M:%S')
            except ValueError:
                inicio_dt = fim_dt = 'N/A'
        
        # Verificar se está aberta às 01:15 PM
        aberta = esta_aberta(horarios)
        
        # Armazenar os dados
        farmacias.append({
            'nome': nome,
            'tipo': tipo,
            'morada': morada,
            'telefone': telefone,
            'horarios': horarios,
            'servicos': servicos,
            'website': website,
            'link_mapa': link_mapa,
            'coordenadas': coordenadas,
            'timestamp_inicio': inicio_dt,
            'timestamp_fim': fim_dt,
            'aberta_13_15': 'Sim' if aberta else 'Não'
        })

# Filtrar farmácias abertas às 01:15 PM
farmacias_abertas = [f for f in farmacias if f['aberta_13_15'] == 'Sim']

# Preparar dados para a tabela
tabela = []
for f in farmacias_abertas:
    tabela.append([
        f['nome'],
        f['tipo'],
        f['morada'],
        f['telefone'],
        ', '.join(f['horarios']),
        ', '.join(f['servicos']) if f['servicos'] else 'N/A',
        f['website'],
        f['link_mapa'],
        f['coordenadas'],
        f['timestamp_inicio'],
        f['timestamp_fim']
    ])

# Definir cabeçalhos da tabela
headers = ['Nome', 'Tipo', 'Morada', 'Telefone', 'Horários', 'Serviços', 'Website', 'Link Mapa', 'Coordenadas', 'Início', 'Fim']

# Exibir a tabela com tabulate
print("\nFarmácias abertas às 01:15 PM WEST (30/05/2025):")
print(tabulate(tabela, headers=headers, tablefmt='grid'))

# Informar o número de farmácias encontradas
print(f"\nTotal de farmácias abertas: {len(farmacias_abertas)}")


Farmácias abertas às 01:15 PM WEST (30/05/2025):
+------------------------+------------------+---------------------------------------------------------------------------------+-------------+----------------+------------------------------------------------------------------------------+-------------------------------------------+-------------+---------------+---------------------+---------------------+
| Nome                   | Tipo             | Morada                                                                          | Telefone    | Horários       | Serviços                                                                     | Website                                   | Link Mapa   |   Coordenadas | Início              | Fim                 |
+========================+==================+=================================================================================+=============+================+==============================================================================+=